In [1]:
from API_lib import *

import time
import sys
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display
from ipywidgets import HBox, VBox, FloatSlider
import ipywidgets as widgets

In [2]:
%matplotlib widget
plt.ioff()
plt.clf()

In [41]:
level = 1 # Levels 1 and 2 are single player

In [42]:
info1 = declare_player(level)
pid1 = info1['private_id']

In [43]:
info1

{'context': 'Save private ID for later reference; use `status` endpoint to track further progress',
 'private_id': '116c23542a5d4373',
 'status': 'Level 1 game countdown to your serve'}

In [44]:
if level < 3:
    # single player
    state = call('status', id_code=pid1)
else:
    # multiplayer -- wait for another to join in lobby
    while True:
        time.sleep(0.3)
        state = call('status', id_code=pid1)
        if 't' in state and state['t'] < 0:
            # ready to move forward with 2 players
            break

In [45]:
# check that everything is ready and see countdown time to begin
state

{'ball': [-0.340299401, -24.8860757647],
 'court length': 49.7721515294,
 'court width': 22.8817969179,
 'max_paddle_move': 0.55,
 'status': 'Level 1 game countdown to your serve',
 't': -8.777557373,
 'your public ID': '42ac09d2774b5cc6',
 'your_paddle': [[-1.3733469738, -24.8860757647],
  [0.6927481717, -24.8860757647]]}

In [46]:
print("Game starts in {:.2f} seconds, execute the interactive plot below".format(abs(state['t'])))

Game starts in 8.78 seconds, execute the interactive plot below


## Keys "D" "F" "." "G" "H" work as the buttons inside the next cell, respectively

**The plot only refreshes if you take one of these actions**

In [47]:
xdim = state['court width']
ydim = state['court length']
paddle_lo, paddle_hi = state['your_paddle']
max_dx = state['max_paddle_move']
ball = state['ball']
    
def draw_court(ax):
    ax.plot((-xdim/2,xdim/2), (-ydim/2,-ydim/2), 'r-', alpha=0.1);
    ax.plot((-xdim/2,xdim/2), (ydim/2,ydim/2), 'r-', alpha=0.1);
    ax.plot((-xdim/2,-xdim/2), (-ydim/2,ydim/2), 'r-', alpha=0.1);
    ax.plot((xdim/2,xdim/2), (-ydim/2,ydim/2), 'r-', alpha=0.1);
    
fig, ax = plt.subplots(figsize=(4,6))
#plt.box(False)
ax.autoscale(enable=False) #, tight=False)
draw_court(ax)
fig.tight_layout()
ax.set_xlim(-1-xdim/2, xdim/2+1)
ax.set_ylim(-1-ydim/2, ydim/2+1)
ax.set_aspect('equal')
ax.axis('off')
plt.suptitle("t = 0", y=1.)
#txt_handle = ax.text(1,75, "t = 0")
#plt.show()
fig.canvas.draw()
fig.canvas.flush_events()
pA_handle = ax.plot((state['your_paddle'][0][0], state['your_paddle'][1][0]),
                    (-ydim/2-0.2,-ydim/2-0.2), 'k-', lw=5)[0]
if 'opp_paddle' in state:
    pB_handle = ax.plot((state['opp_paddle'][0][0], state['opp_paddle'][1][0]),
                    (ydim/2+0.2,ydim/2+0.2), 'k-', lw=5)[0]
ball_handle = ax.plot(ball[0], ball[1], 'ko', ms=5)[0]

buttonLL = widgets.Button(description="LL")
buttonRR = widgets.Button(description="RR")
buttonV = widgets.Button(description="view")
buttonL = widgets.Button(description="L")
buttonR = widgets.Button(description="R")

def moveLL(b):
    update(call('move/{:.4f}'.format(-max_dx), pid1))
    
def moveRR(b):
    update(call('move/{:.4f}'.format(max_dx), pid1))

def moveL(b):
    update(call('move/{:.4f}'.format(-max_dx/2), pid1))
    
def moveR(b):
    update(call('move/{:.4f}'.format(max_dx/2), pid1))

def view(b):
    update(call('status', pid1))

buttonL.on_click(moveL)
buttonR.on_click(moveR)
buttonLL.on_click(moveLL)
buttonRR.on_click(moveRR)
buttonV.on_click(view)

def press(event):
    sys.stdout.flush()
    if event.key == 'd':
        update(call('move/{:.4f}'.format(-max_dx), pid1))
    elif event.key == 'f':
        update(call('move/{:.4f}'.format(-max_dx/2), pid1))
    elif event.key == 'g':
        update(call('move/{:.4f}'.format(max_dx/2), pid1))
    elif event.key == 'h':
        update(call('move/{:.4f}'.format(max_dx), pid1))
    elif event.key == '.':
        update(call('status', pid1))

fig.canvas.mpl_connect('key_press_event', press)

def update(s):
    if 'won' in s['status']:
        print(s)
        sys.stdout.flush()
    else:        
        ball_handle.set_data(s['ball'][0], s['ball'][1])
        pA_handle.set_data((s['your_paddle'][0][0], s['your_paddle'][1][0]), (-ydim/2-0.2,-ydim/2-0.2))
        if 'opp_paddle' in s:
            pB_handle.set_data((s['opp_paddle'][0][0], s['opp_paddle'][1][0]), (ydim/2+0.2,ydim/2+0.2))
        #txt_handle.set_text("t = %.2f, (%.2f, %.2f)" % (s['t'], s['ball'][0], s['ball'][1]))
        plt.suptitle("t = %.2f, (%.2f, %.2f)" % (s['t'], s['ball'][0], s['ball'][1]), y=1.)
        fig.canvas.draw();
        fig.canvas.flush_events();
        
buttons = HBox([buttonLL, buttonL, buttonV, buttonR, buttonRR])
VBox([buttons, fig.canvas])

In [13]:
# call this afterwards to see outcome
call('status', id_code=pid1)

{'context': 'Use `request_game` endpoint first',
 'status': 'Player has not joined lobby'}